In [1]:
import os
os.chdir("../../")

from src.chroma_store import initialize_vectorstore, load_documents_from_dir
from src.chunking_strategies import chunk_by_semantic, chunk_by_recursive_split

In [2]:

from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
)

In [3]:
# Load the documents from the data directory.
documents = load_documents_from_dir("data/content")

chunks = chunk_by_recursive_split(documents, chunk_size=400)
# chunks = chunk_by_semantic(documents)


--INFO-- Loading documents from data/content
--INFO-- Loaded 1 documents
Split 1 documents into 51 chunks.
Time Tracking: Advisor shall provide the Company with a written report, in a format acceptable by the Company, setting forth the number of hours in which he provided the Services, on a daily basis, as well as an aggregated monthly report at the last day of each calendar month.
{'source': 'data/content/Robinson Advisory.docx', 'start_index': 2311}


### Semantic Chunking
#### Gradient
In this method, the gradient of distance is used to split chunks along with the percentile method. This method is useful when chunks are highly correlated with each other or specific to a domain e.g. legal or medical. The idea is to apply anomaly detection on gradient array so that the distribution become wider and easy to identify boundaries in highly semantic data.

In [6]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

text_splitter = SemanticChunker(
    OpenAIEmbeddings()
)

In [8]:
docs = text_splitter.create_documents([document.page_content for document in documents])
print(docs[0].page_content)

2024-07-10 07:58:56 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


- 2-

ADVISORY SERVICES AGREEMENT

This Advisory Services Agreement is entered into as of June 15th, 2023 (the “Effective Date”), by and between Cloud Investments Ltd., ID 51-426526-3, an Israeli company (the "Company"), and Mr. Jack Robinson, Passport Number 780055578, residing at 1 Rabin st, Tel Aviv, Israel, Email: jackrobinson@gmail.com ("Advisor"). Whereas,	Advisor has expertise and/or knowledge and/or relationships, which are relevant to the Company’s business and the Company has asked Advisor to provide it with certain Advisory services, as described in this Agreement; and

Whereas, 	Advisor has agreed to provide the Company with such services, subject to the terms set forth in this Agreement.


In [7]:
vectorstore = initialize_vectorstore(docs)

2024-07-07 22:11:48 - src.chroma_store - INFO - Clearing out the chroma database.
2024-07-07 22:11:48 - src.chroma_store - INFO - Creating a new chroma database.
2024-07-07 22:11:49 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-07-07 22:11:51 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


### Initialize retriever

In [ ]:
similarity_threshold = 0.5
similarity_count = 20
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                      search_kwargs={'score_threshold': similarity_threshold,
                                                      "k": similarity_count})